In [ ]:
# map_generator.ipynb
# RPGシミュレーター - マップ生成ツール
#
# このノートブックはRPGシミュレーター用の大きなマップを生成し、
# JSONファイルとして保存します。

import os
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

# 必要なクラスと関数を読み込む
%run notebooks/map_system.ipynb

# データディレクトリの確認
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"'{data_dir}'ディレクトリを作成しました。")

# マップサイズの指定
width = 80   # マップの幅
height = 60  # マップの高さ

print(f"サイズ {width}x{height} のマップを生成します...")

# 新しい大きな地図を作成
game_map = GameMap(f"広大な王国", width, height)

# 基本的な地形を設定
for row in range(game_map.height):
    for col in range(game_map.width):
        # デフォルトは平地
        game_map.set_tile(row, col, TILE_EMPTY)

# 森林地帯の生成
for _ in range(width * height // 12):
    row = random.randint(0, height - 1)
    col = random.randint(0, width - 1)
    
    # 森を塊で生成
    forest_size = random.randint(3, 8)
    for r in range(max(0, row - forest_size // 2), min(height, row + forest_size // 2 + 1)):
        for c in range(max(0, col - forest_size // 2), min(width, col + forest_size // 2 + 1)):
            if random.random() < 0.7:  # 70%の確率で森を配置
                game_map.set_tile(r, c, TILE_FOREST)

# 山脈の生成
for _ in range(width * height // 25):
    row = random.randint(0, height - 1)
    col = random.randint(0, width - 1)
    
    # 山を塊で生成
    mountain_size = random.randint(2, 6)
    for r in range(max(0, row - mountain_size // 2), min(height, row + mountain_size // 2 + 1)):
        for c in range(max(0, col - mountain_size // 2), min(width, col + mountain_size // 2 + 1)):
            if random.random() < 0.6:  # 60%の確率で山を配置
                game_map.set_tile(r, c, TILE_MOUNTAIN)

# 川の生成（複数の川）
for _ in range(3):  # 3本の川
    river_start_col = random.randint(5, width - 5)
    river_col = river_start_col
    
    # 川の始点をランダムに選択（上端か下端）
    if random.random() < 0.5:
        # 上から下
        for row in range(height):
            river_width = random.randint(1, 2)
            for i in range(river_width):
                col_pos = min(width - 1, max(0, river_col + i - river_width // 2))
                game_map.set_tile(row, col_pos, TILE_WATER)
            
            # 川の流れをランダムに曲げる
            river_col += random.randint(-1, 1)
            river_col = max(1, min(width - 2, river_col))
    else:
        # 下から上
        for row in range(height - 1, -1, -1):
            river_width = random.randint(1, 2)
            for i in range(river_width):
                col_pos = min(width - 1, max(0, river_col + i - river_width // 2))
                game_map.set_tile(row, col_pos, TILE_WATER)
            
            # 川の流れをランダムに曲げる
            river_col += random.randint(-1, 1)
            river_col = max(1, min(width - 2, river_col))

# 草原の生成
for row in range(height):
    for col in range(width):
        if game_map.get_tile(row, col) == TILE_EMPTY and random.random() < 0.3:
            game_map.set_tile(row, col, TILE_GRASS)

# 主要道路の生成（十字の道）
main_road_row = random.randint(height // 4, 3 * height // 4)
for col in range(width):
    if game_map.get_tile(main_road_row, col) != TILE_WATER:
        game_map.set_tile(main_road_row, col, TILE_ROAD)

main_road_col = random.randint(width // 4, 3 * width // 4)
for row in range(height):
    if game_map.get_tile(row, main_road_col) != TILE_WATER:
        game_map.set_tile(row, main_road_col, TILE_ROAD)

# 道路のジャンクションを確保（町の位置を確定）
junction_row = main_road_row
junction_col = main_road_col

# 始まりの村を配置
start_town = Location(
    name="始まりの村",
    description="この地方で最も古い村。多くの冒険者がここから旅立つ。",
    loc_type="town",
    row=junction_row,
    col=junction_col,
    events=[
        {
            "type": "shop",
            "name": "村の雑貨屋",
            "description": "日用品や冒険の道具を売っている"
        },
        {
            "type": "inn",
            "name": "村の宿屋",
            "description": "安価で泊まれる宿。体力が全回復する"
        }
    ]
)
game_map.add_location(start_town)

# その他の町を道路沿いに配置
town_names = ["森の町", "川辺の集落", "交易都市", "南の町", "砂漠の町", "山麓の村", "東の港町"]
road_positions = []

# 道路沿いの位置を収集
for row in range(height):
    for col in range(width):
        if (game_map.get_tile(row, col) == TILE_ROAD and 
            (row != junction_row or col != junction_col) and  # 始まりの村の位置を除外
            (abs(row - junction_row) > 10 or abs(col - junction_col) > 10)):  # 距離を確保
            road_positions.append((row, col))

# 町を配置
random.shuffle(road_positions)
for i, town_name in enumerate(town_names):
    if i >= len(road_positions):
        break
        
    row, col = road_positions[i]
    town = Location(
        name=town_name,
        description=f"道沿いに広がる{town_name}。旅人で賑わっている。",
        loc_type="town",
        row=row,
        col=col,
        events=[
            {
                "type": "shop",
                "name": f"{town_name}の店",
                "description": "様々な商品が並んでいる"
            },
            {
                "type": "inn",
                "name": f"{town_name}の宿",
                "description": "旅の疲れを癒せる宿屋"
            }
        ]
    )
    game_map.add_location(town)

# ダンジョンを森や山の近くに配置
dungeon_names = ["古代遺跡", "魔物の洞窟", "忘れられた鉱山", "禁断の森", "地下迷宮"]
dungeon_candidates = []

# ダンジョン候補地を探す（森や山の近く）
for row in range(height):
    for col in range(width):
        if game_map.get_tile(row, col) not in [TILE_WATER, TILE_ROAD]:
            # 周囲に森や山があるか確認
            near_feature = False
            for r in range(max(0, row-2), min(height, row+3)):
                for c in range(max(0, col-2), min(width, col+3)):
                    if game_map.get_tile(r, c) in [TILE_FOREST, TILE_MOUNTAIN]:
                        near_feature = True
                        break
            
            if near_feature:
                # 既存のロケーションから離れているか確認
                too_close = False
                for (loc_row, loc_col) in game_map.locations:
                    dist = ((row - loc_row)**2 + (col - loc_col)**2)**0.5
                    if dist < 10:  # 最低距離
                        too_close = True
                        break
                
                if not too_close:
                    dungeon_candidates.append((row, col))

# ダンジョンを配置
random.shuffle(dungeon_candidates)
for i, dungeon_name in enumerate(dungeon_names):
    if i >= len(dungeon_candidates):
        break
        
    row, col = dungeon_candidates[i]
    dungeon = Location(
        name=dungeon_name,
        description=f"不気味な雰囲気が漂う{dungeon_name}。強力なモンスターが潜んでいるという噂だ。",
        loc_type="dungeon",
        row=row,
        col=col,
        events=[
            {
                "type": "battle",
                "name": f"{dungeon_name}の主",
                "description": f"{dungeon_name}の奥でモンスターに遭遇した！"
            }
        ]
    )
    game_map.add_location(dungeon)

# 王城を配置（もっとも遠い場所に）
max_dist = 0
castle_pos = None

for row in range(height):
    for col in range(width):
        if game_map.get_tile(row, col) not in [TILE_WATER, TILE_MOUNTAIN]:
            # 始まりの村からの距離を計算
            dist = ((row - junction_row)**2 + (col - junction_col)**2)**0.5
            
            # より遠い位置を選択
            if dist > max_dist:
                # 既存のロケーションから離れているか確認
                too_close = False
                for (loc_row, loc_col) in game_map.locations:
                    loc_dist = ((row - loc_row)**2 + (col - loc_col)**2)**0.5
                    if loc_dist < 15:  # 最低距離
                        too_close = True
                        break
                
                if not too_close:
                    max_dist = dist
                    castle_pos = (row, col)

# 王城を追加
if castle_pos:
    castle_row, castle_col = castle_pos
    
    # 城の周りに道を作る
    for r in range(max(0, castle_row-1), min(height, castle_row+2)):
        for c in range(max(0, castle_col-1), min(width, castle_col+2)):
            if game_map.get_tile(r, c) != TILE_WATER:
                game_map.set_tile(r, c, TILE_ROAD)
    
    castle = Location(
        name="王都",
        description="この地方を治める王の城。立派な建物が建っている。",
        loc_type="castle",
        row=castle_row,
        col=castle_col,
        events=[
            {
                "type": "quest",
                "name": "王様の依頼",
                "description": "王様から魔物退治の依頼を受けた"
            }
        ]
    )
    game_map.add_location(castle)

# マップ生成完了メッセージ
print(f"マップ生成完了: {len(game_map.locations)}個のロケーションを配置")

# マップをJSONに保存
map_path = os.path.join(data_dir, 'large_world_map.json')

# GameMapクラスにsave_to_file_extendedメソッドを追加（map_system.ipynbに既にあればこの部分は不要）
def save_to_file_extended(self, filepath: str) -> bool:
    """マップデータをJSONファイルに詳細に保存
    
    Args:
        filepath: 保存先のファイルパス
            
    Returns:
        保存に成功したかどうか
    """
    try:
        # 基本データ
        data = {
            'version': '1.1',
            'name': self.name,
            'width': self.width,
            'height': self.height,
            'tiles': self.tiles.tolist(),
            'locations': {}
        }
        
        # ロケーションデータを詳細に保存
        for (row, col), loc in self.locations.items():
            location_data = {
                'name': loc.name,
                'description': loc.description,
                'type': loc.loc_type,
                'row': row,
                'col': col,
                'visited': loc.visited,
                'events': loc.events
            }
            data['locations'][f"{row},{col}"] = location_data
        
        # 人間が読みやすいようにJSON形式で保存
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"マップを保存しました: {filepath}")
        return True
        
    except Exception as e:
        print(f"マップの保存中にエラーが発生しました: {e}")
        return False

# すでにGameMapクラスにメソッドが追加されていない場合にのみ追加
if not hasattr(GameMap, 'save_to_file_extended'):
    GameMap.save_to_file_extended = save_to_file_extended

# マップを保存
game_map.save_to_file_extended(map_path)

#日本語対策が必要であれば


# マップの可視化（大きなマップなのでサムネイル表示）
plt.figure(figsize=(12, 10))

# マップをカラーコードとして表示
cmap = plt.matplotlib.colors.ListedColormap(
    [TILE_COLORS[i] for i in range(max(TILE_COLORS.keys()) + 1)]
)
plt.imshow(game_map.tiles, cmap=cmap, interpolation='nearest')

# ロケーションに印をつける
for (row, col), location in game_map.locations.items():
    if location.loc_type == 'town':
        plt.plot(col, row, 'bo', markersize=8)  # 青丸で町を表示
        plt.text(col, row+1, location.name, fontsize=8, ha='center')
    elif location.loc_type == 'castle':
        plt.plot(col, row, 'yo', markersize=10)  # 黄丸で城を表示
        plt.text(col, row+1, location.name, fontsize=8, ha='center')
    elif location.loc_type == 'dungeon':
        plt.plot(col, row, 'ro', markersize=8)  # 赤丸でダンジョンを表示
        plt.text(col, row+1, location.name, fontsize=8, ha='center')

plt.title(f"マップ: {game_map.name} ({game_map.width}x{game_map.height})")
plt.tight_layout()
plt.show()

# マップの統計情報表示
print("\n=== マップ統計情報 ===")
print(f"マップ名: {game_map.name}")
print(f"サイズ: {game_map.width}x{game_map.height} ({game_map.width * game_map.height}マス)")

# タイルタイプの集計
tile_counts = {}
for row in range(game_map.height):
    for col in range(game_map.width):
        tile = game_map.get_tile(row, col)
        tile_counts[tile] = tile_counts.get(tile, 0) + 1

print("\n地形分布:")
for tile, count in sorted(tile_counts.items()):
    percentage = (count / (game_map.width * game_map.height)) * 100
    print(f"  {TILE_NAMES.get(tile, f'タイプ{tile}')}： {count}マス ({percentage:.1f}%)")

# ロケーション情報
location_types = {}
for _, location in game_map.locations.items():
    location_types[location.loc_type] = location_types.get(location.loc_type, 0) + 1

print("\nロケーション:")
for loc_type, count in sorted(location_types.items()):
    print(f"  {loc_type}: {count}箇所")
print(f"  合計: {len(game_map.locations)}箇所")

# 全ロケーションリスト
print("\n全ロケーション一覧:")
for (row, col), location in sorted(game_map.locations.items(), key=lambda x: x[1].name):
    print(f"  {location.name} ({location.loc_type}) - 座標: ({row}, {col})")

print("\nJSONファイル生成完了！RPGゲームが使用できる大きなマップデータが保存されました。")